** 1. Downloading and storing 8000 tweets (6491) **


In [1]:
import twython as tw
import pandas as pd
import pyprind
import os

#twitter api credentials
CONSUMER_KEY = "qGWa0auDtpVJXx1LaZQspsVWe"
CONSUMER_SECRET = "scSr5x4R1u7w7KAoQ47J8xG1J0nUozhpowzzjNPO2DYuD89plg"
OAUTH_TOKEN = "846031238813171713-5ywVPWAEPNDBlVnO6ReosjvAcTn5K3v"
OAUTH_TOKEN_SECRET = "1itTLCn9ytm6L0UNeHWObgaofsk4qE62cHyHvxSa1nBRH"
twitter = tw.Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


df = pd.DataFrame()


i = 0
pbar = pyprind.ProgBar(8000 - i)

maxid = 0
#maxid = 849024394470899713

while i<8000:
    try:
        if maxid == 0 :
            search = twitter.search(q='#sarcasm -filter:retweets AND -filter:replies', result_type='recent', lang='en', count=100)
        else:
            search = twitter.search(q='#sarcasm -filter:retweets AND -filter:replies', result_type='recent', lang='en', count=100, max_id=maxid)


        tweets = search['statuses']

        tid = ""
        for tweet in tweets:
            df = df.append([[tweet['text'].replace("\n", "")]], ignore_index=True)
            i += 1
            tid = tweet['id']
            #print tweet['id'], tweet['text'], '\n\n\n'
            pbar.update()
        try:
            maxid = int(tid)
        except:
            print tid
    except:
        break

print maxid
print i
df.columns = ['tweet']
df.to_csv('./training_set_f.csv', index=False, encoding='utf-8')

0%                          100%
[############                  ] | ETA: 00:01:02

854867673993359360
3249


** 2. Eliminating duplicates from downloaded tweets (3264 -> 3017) **


In [2]:
!sort -u training_set_f.csv > training_set_f2.csv

In [3]:
!grep -r ".*" training_set_f2.csv | wc -l

3017


** 3. Extracting implicit phrases**


In [6]:
import re
from nltk.util import ngrams
from nltk.tokenize import TweetTokenizer

def preprocessor(text):
    #text = re.sub("#[^\s+]+", "", text)
    #return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()
    tt = TweetTokenizer()
    tokenized = tt.tokenize(text)
    tokenized_f = []
    for token in tokenized:
        if any(special in token for special in ["#", ":)", ":-)", ":(", ":/", ":P"]):
            print token
        else:
            tokenized_f.append(token)
    return tokenized_f

def extract_n_grams(sentence, lowerbound=3, upperbound=10):
    lm = {n:dict() for n in range(lowerbound, upperbound + 1)}
    for n in range(lowerbound, upperbound + 1):
        ngram = ngrams(sentence, n)
        for item in ngram:
            lm[n][item] = lm[n].get(item, 0) + 1
    return lm
    
niz1 = "Mihael nezna programirati, pa pishe' gluposti i sramoti se:) #sarcasm"
preprocessor(niz1)
print extract_n_grams(preprocessor(niz1))[3]

:)
#sarcasm
:)
#sarcasm
{(u'nezna', u'programirati', u','): 1, (u"'", u'gluposti', u'i'): 1, (u'gluposti', u'i', u'sramoti'): 1, (u'pa', u'pishe', u"'"): 1, (u',', u'pa', u'pishe'): 1, (u'pishe', u"'", u'gluposti'): 1, (u'i', u'sramoti', u'se'): 1, (u'Mihael', u'nezna', u'programirati'): 1, (u'programirati', u',', u'pa'): 1}


In [7]:
import re
from nltk.util import ngrams
import pyprind


def preprocessor(text):
    text = re.sub("#[^\s+]+", "", text)
    return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()

def extract_n_grams(phrases, sentence, lowerbound=3, upperbound=10):
    for n in range(lowerbound, upperbound + 1):
        ngram = ngrams(sentence, n)
        for item in ngram:
            phrases[n][item] = phrases[n].get(item, 0) + 1
    return phrases
#Intialiaze upper and lower bound
lowerbound = 3
upperbound = 10

#initialize dict with phrases
phrases = {n:dict() for n in range(lowerbound, upperbound + 1)}

def extract_n_phrases(training_set_file, progress_bar=True):
    if progress_bar:
        tset_len = sum(1 for row in open(training_set_file, 'r'))
    with open(training_set_file, 'r') as training_set:
        if progress_bar:
            pbar = pyprind.ProgBar(tset_len)
        for tweet in training_set:
            extract_n_grams(phrases, preprocessor(tweet))
            #print tweet, preprocessor(tweet)
            #print extract_n_grams(phrases, preprocessor(tweet))[3]
            #for e in extract_n_grams(phrases, preprocessor(tweet))[3]:
            #    print extract_n_grams(phrases, preprocessor(tweet))[3][e]
            #    break
            if progress_bar:
                pbar.update()

extract_n_phrases('./training_set_f2.csv')

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


** 4. Extract phrases that occur at least thrice**

In [8]:
phrases_filtered = {n:dict() for n in range(lowerbound, upperbound + 1)}
for n_gram in range(lowerbound, upperbound + 1):
    for phrase in phrases[n_gram]:
        if phrases[n_gram][phrase] >= 3:
            phrases_filtered[n_gram][phrase] = phrases[n_gram][phrase]

** 5. Store n-grams to file **

In [9]:
import pickle

output_phrases = open('./grams/n_grams.pkl', 'wb')
pickle.dump(phrases_filtered, output_phrases)
output_phrases.close()

** 6. Convert n-grams to phrases**

In [10]:
phrases_converted = []
for n_gram in range(lowerbound, upperbound + 1):
    for phrase in phrases_filtered[n_gram]:
        key = ' '.join(phrase)
        phrases_converted.append(key)

In [14]:
df = pd.DataFrame()
for phrase in phrases_converted:
    df = df.append([phrase], ignore_index=True)

df.columns = ['phrase']
df.head(10)


,phrase
0,don t say
1,it will be
2,looking forward to
3,t see that
4,Thank you for
5,I really want
6,you don t
7,look like a
8,if I m
9,love it when


In [11]:
import pickle

print "Number of phrases:", len(phrases_converted)

output_phrases = open('./grams/phrases.pkl', 'wb')
pickle.dump(phrases_converted, output_phrases)
output_phrases.close()

Number of phrases: 505
